In [2]:
from byu.data.datasets.heatmap_3d_dataset import Heatmap3dDataset
from omegaconf import OmegaConf
import logging
from tqdm import tqdm
from itkwidgets import view
import numpy as np
import copy
from matplotlib import pyplot as plt
import pandas as pd
from torch.nn import functional as F

In [36]:
from omegaconf import OmegaConf

logging.basicConfig(level=logging.INFO)

yaml_str = """

env:
    data_dir: /home/dangnh36/datasets/.comp/byu/

cv:
    strategy: skf4_rd42
    num_folds: 4
    fold_idx: 1
    train_on_all: True

loader:
    num_workers: 1

data:
    label_fname: gt_v2  # gt | gt_v2 | gt_v3 | all_gt | all_gt_v3
    patch_size: [224, 448, 448]
    start: [0, 0, 0]
    overlap: [0, 0, 0]
    border: [0, 0, 0]
    
    sigma: 0.2
    fast_val_workers: 5
    fast_val_prefetch: 5
    io_backend: cv2  # cv2, cv2_seq, npy, cache
    crop_outside: True
    ensure_fg: True
    label_smooth: [0.0,1.0]
    filter_rule: null  # null | eq1 | le1
    
    
    sampling:
      method: pre_patch  # pre_patch | rand_crop
      pre_patch:
          fg_max_dup: 1
          bg_ratio: 0.0
          bg_from_pos_ratio: 0.01
          overlap: [0, 0, 0]
      rand_crop:
          random_center: True
          margin: 0.25
          auto_correct_center: True
          pos_weight: 1
          neg_weight: 1
    
    transform:
        resample_mode: trilinear # F.grid_sample() mode
        target_spacing: [16,16,16]
        heatmap_mode: gaussian
        heatmap_stride: [16,16,16]
        heatmap_same_sigma: False
        heatmap_same_std: True
        lazy: True
        device: null
    aug:
        enable: True
        zoom_prob: 1.0
        zoom_range: [[0.6, 1.2], [0.6, 1.2], [0.6, 1.2]]  # (X, Y, Z) or (H, W, D)
        # affine1
        affine1_prob: 0.0
        affine1_scale: 0.3  # max_skew_xy = 1.3 / 0.7 = 1.86
        # affine2
        affine2_prob: 1.0
        affine2_rotate_xy: 15 # degrees
        affine2_scale: 0.3  # max_skew_xy = 1.3 / 0.7 = 1.86
        affine2_shear: 0.3
    
        rand_shift: False  # only used in rand_crop, very useful if random_center=auto_correct_center=False
    
        # no lazy, can't properly transform points
        grid_distort_prob: 0.0
        smooth_deform_prob: 0.0
    
        intensity_prob: 0.5
        smooth_prob: 0.0
        hist_equalize: False
        downsample_prob: 0.2
        coarse_dropout_prob: 0.0
    
        # MIXER
        mixup_prob: 0.0
        cutmix_prob: 0.0
        mixer_alpha: 1.0
        mixup_target_mode: max
    
    tta:
        # enable: [zyx]
        enable: [zyx, zxy, zyx_x, zyx_y]
        # enable: [zyx, zyx_x, zyx_y, zyx_z, zyx_xy, zyx_xz, zyx_yz, zyx_xyz, zxy, zxy_x, zxy_y, zxy_z, zxy_xy, zxy_xz, zxy_yz, zxy_xyz]
"""
global_cfg = OmegaConf.create(yaml_str)
dataset = Heatmap3dDataset(global_cfg, stage="train")

/home/dangnh36/projects/.comp/byu/yagm/src/yagm/transforms/keypoints/helper.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  crop_start = torch.tensor(crop_start)[None]
/home/dangnh36/projects/.comp/byu/yagm/src/yagm/transforms/keypoints/helper.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  crop_end = torch.tensor(crop_end)[None] + end_offset
Pre-patching..: 100%|█| 648/648 [00:02<00:00, 


In [18]:
print("LEN:", len(dataset))
for i in tqdm(range(0, 2, 1)):
    sample = dataset[i]
    for k, v in sample.items():
        print(f"{k} -> {type(v)} {v.dtype} {v.shape}")
    print(sample["image"].min(), sample["image"].max())
    print(sample["heatmap"].min(), sample["heatmap"].max())
    print("----------------\n")

LEN: 23040


/opt/conda/lib/python3.11/site-packages/monai/transforms/spatial/array.py:497: UserWarning: `data_array` is not of type MetaTensor, assuming affine to be identity.
  warnings.warn("`data_array` is not of type MetaTensor, assuming affine to be identity.")


idx -> <class 'torch.Tensor'> torch.int64 torch.Size([])
image -> <class 'monai.data.meta_tensor.MetaTensor'> torch.uint8 torch.Size([1, 224, 448, 448])
heatmap -> <class 'torch.Tensor'> torch.float16 torch.Size([1, 224, 448, 448])


 50%|█████▌     | 1/2 [00:06<00:06,  6.59s/it]

metatensor(0, dtype=torch.uint8) metatensor(254, dtype=torch.uint8)
tensor(0., dtype=torch.float16) tensor(0., dtype=torch.float16)
----------------

idx -> <class 'torch.Tensor'> torch.int64 torch.Size([])
image -> <class 'monai.data.meta_tensor.MetaTensor'> torch.uint8 torch.Size([1, 224, 448, 448])
heatmap -> <class 'torch.Tensor'> torch.float16 torch.Size([1, 224, 448, 448])


100%|███████████| 2/2 [00:12<00:00,  6.50s/it]

metatensor(0, dtype=torch.uint8) metatensor(254, dtype=torch.uint8)
tensor(0., dtype=torch.float16) tensor(0., dtype=torch.float16)
----------------



In [ ]:
sample = dataset[611]

image = (sample['image'][0]).cpu().numpy().astype(np.uint8)
heatmap = sample['heatmap']
print('BEFORE INTERPOLATE:', heatmap.shape, heatmap.min(), heatmap.max())
heatmap = F.interpolate(
            heatmap[None],
            size=None,
            scale_factor=1,
            mode='trilinear',
            align_corners=False,
            recompute_scale_factor=False,
        )[0]
print('AFTER INTERPOLATE:', heatmap.shape, heatmap.dtype, heatmap.min(), heatmap.max())
print(image.shape, image.dtype)
heatmap = (heatmap[0] * 255).cpu().numpy().astype(np.uint8)


# view(image = image,
#     label_image=heatmap,
#     label_blend = 1.0,
#     cmap = 'Grayscale',
# )

view(image = heatmap,
    cmap = 'Grayscale',
)

In [4]:
sample = dataset[611]
heatmap = sample['heatmap']
heatmap.shape, heatmap.min(), heatmap.max()

/opt/conda/lib/python3.11/site-packages/monai/transforms/spatial/array.py:497: UserWarning: `data_array` is not of type MetaTensor, assuming affine to be identity.
  warnings.warn("`data_array` is not of type MetaTensor, assuming affine to be identity.")


(torch.Size([1, 56, 112, 112]),
 tensor(0., dtype=torch.float16),
 tensor(1., dtype=torch.float16))

In [6]:
import cc3d

In [24]:
componet = cc3d.connected_components((heatmap[0] > 0.5).numpy(), binary_image=True)
print(np.unique(componet))
print(componet.shape)
stats = cc3d.statistics(componet)
# zyx = stats["centroids"][1:]
print(stats)

[0 1]
(56, 112, 112)
{'voxel_counts': array([694084,   8380], dtype=uint32), 'bounding_boxes': [(slice(0, 56, None), slice(0, 112, None), slice(0, 112, None)), (slice(23, 49, None), slice(64, 89, None), slice(34, 59, None))], 'centroids': array([[27.40232594, 55.25025789, 55.6109419 ],
       [35.58997613, 76.18520286, 46.31109785]])}


In [27]:
heatmap[0][componet == 1].shape

torch.Size([8380])